# Python Code Breakdown

The Python code in this Jupyter notebook uses the ArcPy package to create and manipulate a geodatabase. Here's a step-by-step breakdown:

1. **Import Libraries**: The necessary libraries are imported. This includes ArcPy for geographic data manipulation, random for generating random numbers, and datetime for handling date and time data.

2. **Setup Environment**: The environment is set up and a new File Geodatabase named "weather.gdb" is created in the current working directory.

3. **Create Feature Class**: A new point feature class named 'stations' is created within this geodatabase.

4. **Add Fields**: Fields 'date', 'humidity', 'year', and 'date_str' are added to the 'stations' feature class.

5. **Start Edit Session**: An edit session is started and a domain named 'date_str_domain' is created. This domain is then populated with date strings for each day of the year.

6. **Assign Domain**: The 'date_str_domain' is assigned to the 'date_str' field in the 'stations' feature class.

7. **Generate Data**: Random data for 4 years is generated. This includes coordinates, date, humidity, year, and date_str. This data is inserted into the 'stations' feature class.

8. **Stop Edit Session**: The edit session is stopped, changes are saved, and a confirmation message is printed.

The below screen captures shows how the data produced through this notebook could be visualized once published as a feature layer and used in ArcGIS Experience Builder [Chart widget](https://doc.arcgis.com/en/experience-builder/latest/configure-widgets/chart-widget.htm): 

![alt text](resources/experience_builder_chart_1.png)
ArcGIS Experience Builder [chart widget](https://experience.arcgis.com/experience/9fa299ed70df488085a3268961832f72/) example

In [1]:
import arcpy
import random
from datetime import datetime, timedelta
import os
from dateutil.relativedelta import relativedelta, SU  # Import the necessary functions

# Set overwrite output to True
arcpy.env.overwriteOutput = True

# Set local variables
out_folder_path = os.getcwd()  # Use the current notebook directory
out_name = "weather.gdb"

# Execute CreateFileGDB
arcpy.CreateFileGDB_management(out_folder_path, out_name)
print("File Geodatabase created.")

# Set the workspace
arcpy.env.workspace = f"{out_folder_path}/{out_name}"

# Define the feature class parameters
out_path = arcpy.env.workspace
out_name = "stations"
geometry_type = "POINT"
template = "#"
has_m = "DISABLED"
has_z = "DISABLED"
spatial_reference = arcpy.SpatialReference(4326)  # WGS 1984

# Execute CreateFeatureclass 
arcpy.CreateFeatureclass_management(out_path, out_name, geometry_type, template, has_m, has_z, spatial_reference=spatial_reference)
print("Point Feature Class 'stations' created.")

# Add fields
arcpy.AddField_management("stations", "date", "DATE")
arcpy.AddField_management("stations", "humidity", "FLOAT")
arcpy.AddField_management("stations", "year", "SHORT")  # Add a year field
arcpy.AddField_management("stations", "date_str", "TEXT")  # Add a date string field
print("Fields 'date', 'humidity', 'year', and 'date_str' added.")

# Start an edit session
edit = arcpy.da.Editor(arcpy.env.workspace)
edit.startEditing(False, True)
edit.startOperation()

# Create an insert cursor
cursor = arcpy.da.InsertCursor("stations", ["SHAPE@X", "SHAPE@Y", "date", "humidity", "year", "date_str"])

# Create a domain
domain_name = "date_str_domain"
arcpy.CreateDomain_management(arcpy.env.workspace, domain_name, "Date string domain", "TEXT", "CODED")

# Initialize an empty dictionary
date_dict = {}

# Generate a date for each day of the year and add it to the domain
start_date = datetime(1999, 9, 30)  # Start date is 30-Sep-2000 (a leap year)
for i in range(1, 367):  # 365 days in a year
    # Calculate the date
    date = start_date + timedelta(days=i % 366)  # Use modulo to loop back to the start date after 365 days
    
    # Format the date as a string
    date_str = date.strftime("%d-%b")
    
    # Add the date string to the domain
    arcpy.AddCodedValueToDomain_management(arcpy.env.workspace, domain_name, i, date_str)
    # Add the date string to the dictionary
    date_dict.update({date_str:i})

print("Domain 'date_str_domain' created and populated.")

# Stop the edit operation.
edit.stopOperation()

# Stop the edit session and save the changes
edit.stopEditing(True)


# Assign the domain to the date_str field
arcpy.management.AssignDomainToField("stations", "date_str", domain_name)
print("Domain 'date_str_domain' assigned to 'date_str' field.")

# Start an edit session again for the data population
edit.startEditing(False, True)
edit.startOperation()

# Generate random data for 4 years
start_date = datetime(2019, 9, 30) 
for i in range(4 * 365):  # 4 years of daily data
    # Specify the coordinates
    x, y = 35.7265, 33.4813  # Longitude, Latitude
    
    # Calculate the date
    date = start_date + timedelta(days=i)
    
    # Generate a random humidity value between 60 and 100
    humidity = random.uniform(60, 100)
    
    # Calculate the year from the date
    year = date.year

    # Calculate the last day of the week for the date
    last_day_of_week = date + relativedelta(weekday=SU)
    
    
    # Format the last day of the week as a string
    date_formatted = last_day_of_week.strftime("%d-%b")

    # Find the key in the date_dict that corresponds to the formatted date string
    date_str = date_dict[date_formatted]
    
    # Insert the data into the feature class
    cursor.insertRow([x, y, date, humidity, year, date_str])

# Delete the cursor
del cursor

# Stop the edit operation.
edit.stopOperation()

# Stop the edit session and save the changes
edit.stopEditing(True)

print("Data populated.")

File Geodatabase created.
Point Feature Class 'stations' created.
Fields 'date', 'humidity', 'year', and 'date_str' added.
Domain 'date_str_domain' created and populated.
Domain 'date_str_domain' assigned to 'date_str' field.
Data populated.
